<a href="https://colab.research.google.com/github/Chandru-018/Chandrasekhar_INFO5731_FALL2024/blob/main/Karumanchi_Chandrasekhar_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def collect_reviews(movie_url):
    reviews = []
    page_number = 1

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    while len(reviews) < 1000:
        url = f"{movie_url}?ref_=nv_sr_srsg_0&start={page_number * 10 + 1}"
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Failed to retrieve data from {url}: {e}")
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='review-container')

        if not review_elements:
            print("No more reviews found.")
            break

        for review in review_elements:
            review_text = review.find('div', class_='text show-more__control')
            if review_text:
                reviews.append(review_text.get_text(strip=True))

            if len(reviews) >= 1000:
                break

        page_number += 1
        time.sleep(2)

    return reviews

movie_urls = ["https://www.imdb.com/title/tt15327088/reviews", "https://www.imdb.com/title/tt8178634/reviews/","https://www.imdb.com/title/tt7838252/reviews/","https://www.imdb.com/title/tt15327088/reviews/"]



all_reviews = []

for movie_url in movie_urls:
    print(f"Collecting reviews from: {movie_url}")
    reviews = collect_reviews(movie_url)
    all_reviews.extend(reviews)

with open('movie_reviews.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Review"])
    for review in all_reviews[:1000]:
        writer.writerow([review])

print(f"Collected {len(all_reviews)} reviews. Saved to movie_reviews.csv.")


No more reviews found.
No more reviews found.
No more reviews found.
No more reviews found.
Collected 25 reviews. Saved to movie_reviews.csv.


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [3]:
# Write code for each of the sub parts with proper comments.

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download the necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Load the reviews from the CSV file
df = pd.read_csv('movie_reviews.csv')

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to clean the text data
def clean_text(text):
    # (1) Remove noise (special characters and punctuations)
    text = re.sub(r'[^\w\s]', '', text)

    # (2) Remove numbers
    text = re.sub(r'\d+', '', text)

    # (3) Remove stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])

    # (4) Lowercase all texts
    text = text.lower()

    # (5) Stemming
    text = ' '.join([stemmer.stem(word) for word in text.split()])

    # (6) Lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

# Apply the cleaning function to the reviews
df['Cleaned_Review'] = df['Review'].apply(clean_text)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_movie_reviews.csv', index=False)

print("Cleaned data saved to 'cleaned_movie_reviews.csv'.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Cleaned data saved to 'cleaned_movie_reviews.csv'.


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import nltk
import spacy
from nltk import pos_tag, ne_chunk, word_tokenize
from nltk.tree import Tree
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

df = pd.read_csv('cleaned_movie_reviews.csv')

nlp = spacy.load('en_core_web_sm')

noun_count = 0
verb_count = 0
adj_count = 0
adv_count = 0

def pos_analysis(text):
    global noun_count, verb_count, adj_count, adv_count
    tokens = nltk.word_tokenize(text)
    tagged = pos_tag(tokens)

    for word, tag in tagged:
        if tag.startswith('N'):
            noun_count += 1
        elif tag.startswith('V'):
            verb_count += 1
        elif tag.startswith('J'):
            adj_count += 1
        elif tag.startswith('R'):
            adv_count += 1

    return tagged

def parse_sentences(text):
    doc = nlp(text)

    for sent in doc.sents:
        print("\nConstituency Parsing Tree:")
        parse_tree = ne_chunk(pos_tag(word_tokenize(sent.text)))
        parse_tree.pretty_print()

        print("\nDependency Parsing Tree:")
        print(sent.text)
        for token in sent:
            print(f'{token.text} --> {token.dep_} --> {token.head.text}')

def named_entity_recognition(text):
    doc = nlp(text)
    entity_count = {}

    for ent in doc.ents:
        entity_count[ent.label_] = entity_count.get(ent.label_, 0) + 1

    return entity_count

for review in df['Cleaned_Review']:
    pos_analysis(review)
    parse_sentences(review)
    entities = named_entity_recognition(review)

print("\nPOS Counts:")
print(f"Nouns: {noun_count}, Verbs: {verb_count}, Adjectives: {adj_count}, Adverbs: {adv_count}")

print("\nNamed Entities Count:")
for entity, count in entities.items():
    print(f"{entity}: {count}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.



Constituency Parsing Tree:
                                                                                                                                                                                      S                                                                                                                                                                                               
    __________________________________________________________________________________________________________________________________________________________________________________|__________________________________________________________________________________________________________________________________________________________________________________________      
open/JJ end/NN scene/NN kantara/VBP noth/DT like/IN youv/NNS seen/VBN way/NN wrote/VBD mytholog/NN background/NN good/JJ limit/NN open/JJ scene/NN climax/NN happen/VB overtold/JJ stori/NN rough/NN hero/NN get/VBP troubl/J

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

In [ ]:
Yes, I have submitted in the comment section.

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
This assignment is very interesting and helped me to understand and apply concepts in practical ways. The provided time is sufficient enough to complete the  task.